In [4]:
import json
import re

# Load JSON content
with open('results.json', 'r') as file:
    data = json.load(file)

# Initialize base URL and a list to store the extracted URLs
base_url = 'example.com'
product_urls = []

def extract_product_urls(content, base_url="https://example.com"):
    pattern = re.compile(r'\((\/[^)]+\.html)\)')
    matches = pattern.findall(content)
    return [base_url + match for match in matches]
# Extract URLs from the content
urls = []
for entry in data:
    content = entry.get("content", "")
    urls.extend(extract_product_urls(content))

# Remove duplicates
urls = list(set(urls))

# Print the extracted URLs
for url in urls:
    print(url)
print(len(urls))

https://example.com/sports-nutrition/clear-whey-protein-powder/12081395.html
https://example.com/sports-nutrition/pea-protein-isolate/10530136.html
https://example.com/sports-nutrition/the-whey-sample/12968640.html
https://example.com/sports-nutrition/impact-casein-powder/10798909.html
https://example.com/sports-nutrition/all-in-one-recovery/10530268.html
https://example.com/sports-nutrition/impact-whey-protein-powder/10530943.html
https://example.com/sports-nutrition/soy-protein-isolate/10529701.html
https://example.com/sports-nutrition/clear-vegan-protein/12360400.html
https://example.com/sports-nutrition/vegan-protein-blend/11776868.html
https://example.com/sports-nutrition/clear-whey-diet/13125562.html
https://example.com/sports-nutrition/clear-weight-gainer/13096868.html
https://example.com/sports-nutrition/hydrolysed-whey-protein-powder/10529805.html
https://example.com/sports-nutrition/breakfast-smoothie/13251950.html
https://example.com/sports-nutrition/clear-collagen-protein-p